In [0]:
# import the necessary libraries
import requests
import json
from datetime import datetime
from zoneinfo import ZoneInfo

In [0]:
# mount the adls paths
tiers = ['bronze', 'silver', 'gold']
adls_paths = {tier: f"abfss://{tier}@adlsnews.dfs.core.windows.net" for tier in tiers}
bronze_path = adls_paths['bronze']
silver_path = adls_paths['silver']
gold_path = adls_paths['gold']
dbutils.fs.ls(bronze_path)


[FileInfo(path='abfss://bronze@adlsnews.dfs.core.windows.net/2025-06-25/', name='2025-06-25/', size=0, modificationTime=1750899209000)]

In [0]:
# make the API call
seattle_tz = ZoneInfo("America/Los_Angeles")
now_seattle = datetime.now(seattle_tz)
current_date = now_seattle.strftime('%Y-%m-%d')
url = "https://api.mediastack.com/v1/news"
params = {
    "access_key": "65303e8978fda6cedf226e4eb1ec5dd2",
    "countries": "us,il",
    "languages": "en",
    "date": current_date
}

try:
    response = requests.get(url, params= params)
    response.raise_for_status()
    data = response.json()['data']
    if not data:
        raise ValueError("No data returned from API")
    else:
        path = f"{bronze_path}/{current_date}/latest_news.json"
        json_data = json.dumps(data)
        dbutils.fs.put(path, json_data, overwrite= True)
        print(f"Data written to {path} successfully")
except requests.exceptions.HTTPError as e:
    print(f"HTTP error occurred: {e}")
except ValueError as e:
    print(f"Value error occurred: {e}")
except Exception as e:
    print(f"Error: {e}")



Wrote 16115 bytes.
Data written to abfss://bronze@adlsnews.dfs.core.windows.net//2025-06-25/latest_news.json successfully


In [0]:
# pass the variables as arguments
bronze_output = {
    "bronze_path": bronze_path,
    "silver_path": silver_path,
    "gold_path": gold_path,
    "current_date": current_date,
    "bronze_write_path": path
}
dbutils.jobs.taskValues.set(key= "bronze_output", value= bronze_output)